In [1]:
# from IPython.display import HTML, Image
# HTML('''
#     <script src='https://ajax.googleapis.com/ajax/libs/jquery/3.6.1/jquery.min.js'></script>
#     <script>
#     code_show=false;
#     function code_toggle() {
#         if (code_show){
#         $('div.jp-Editor').show();
#         } else {
#         $('div.jp-Editor').hide();
#     }
#         code_show = !code_show
#     }
#     $( document ).ready(code_toggle);
#     </script>
#     <form action='javascript:code_toggle()'><input type='submit' value='Click here to toggle on/off the raw code.'></form>''')


In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

plt.style.use('style/advocate.mplstyle')
%matplotlib inline
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

The imported file is the Excel spreadsheet provided by the state on 11/28/2022. In excel, the format of all columns containing floats was changed to eliminate commas and the file was saved as CSV.

In [3]:
rh = pd.read_csv('raw data/Road Home Closed with Jurisdiction Census Info 7-19-2022 with ARS.csv', dtype='str')

float_fields =    ['Closing Option', 
                    'TOTAL_CLOSING_AMOUNT',
                    'Total CG Amount',
                    'Total ACG Amunt',
                    'Total Elevation Amount',
                    'Total IMM Amount',
                    'Current Damage Assessment',
                    'Current Damage Assessment - Type 1',
                    'Current Damage Assessment - Type 2',
                    'Damage Type 1 or 2',
                    'Current PSV',
                    'Current Total DOB Amount (no Legal Fees removed)',
                    'Current Legal Fees',
                    'Closing Damage Assessment',
                    'PSV at Closing',
                    'Closing Total DOB Amount',
                    'Difference between Current PSV and Current Damage Assessment Type 2 (col AG - col AC'
                    ]
for field in float_fields:
    rh[field] = rh[field].astype('float64')


In [4]:
# Exclude the row that includes the sum for each field
rh = rh[rh['Census Block'].notnull()]

In [5]:
rh.columns

Index(['Structure Type', 'GIS City', 'GIS State', 'GIS Zip', 'PARISH',
       'Closing Option', 'TOTAL_CLOSING_AMOUNT', 'Total CG Amount',
       'Total ACG Amunt', 'Total Elevation Amount', 'Total IMM Amount',
       'NOLA Planning District Number', 'NOLA Planning District Name',
       'NOLA Neighborhood Number', 'NOLA Neighborhood Name', 'STFID',
       'Census Block', 'Block Group', 'Census Track', 'ORL_CD_NUM',
       'ORL_CD_NAM', 'Current Damage Assessment',
       'Current Damage Assessment - Type 1',
       'Current Damage Assessment - Type 2', 'Damage Type 1 or 2',
       'Current PSV', 'Current Total DOB Amount (no Legal Fees removed)',
       'Current Legal Fees', 'Closing Damage Assessment', 'PSV at Closing',
       'Closed file - PSV Option 1 With No ACG -excluding Sold Home, Inelig, Deceased and files closed with RHEI & IMM only -23,912',
       'Closed File - Option 2/3, Sold Home, Ineligible, Deceased, and Closed with RHEI and IMM Only',
       'Closed with Approved Un

In [6]:
print('Basic stats:')
print(f"Number of records: {len(rh):,.0f}")
print(f"Number of non-null records: {len(rh.dropna(axis=1)):,.0f}")
print(f"Number of records where CG>0 and DA>0: {len(rh[(rh['Current Damage Assessment']>0) & (rh['Total CG Amount']>0)]):,.0f}")
print(f"Total grant amount: ${rh['TOTAL_CLOSING_AMOUNT'].sum():,.2f}")
print(f"Total damage assessment: ${rh['Current Damage Assessment'].sum():,.2f}")
print(f"Total CG amount: ${rh['Total CG Amount'].sum():,.2f}")
print(f"Total ACG amount: ${rh['Total ACG Amunt'].sum():,.2f}")
print(f"Total elevation amount: ${rh['Total Elevation Amount'].sum():,.2f}")
print(f"Total IMM amount: ${rh['Total IMM Amount'].sum():,.2f}")
print(f"Total DOB amount: ${rh['Current Total DOB Amount (no Legal Fees removed)'].sum():,.2f}")
print(f"Total legal fees: ${rh['Current Legal Fees'].sum():,.2f}")


Basic stats:
Number of records: 130,053
Number of non-null records: 130,053
Number of records where CG>0 and DA>0: 122,668
Total grant amount: $9,030,446,583.33
Total damage assessment: $18,313,314,464.07
Total CG amount: $5,767,978,729.72
Total ACG amount: $2,126,950,433.45
Total elevation amount: $942,775,529.66
Total IMM amount: $192,741,890.50
Total DOB amount: $6,026,881,082.18
Total legal fees: $38,267,095.46


In [7]:
pipeline = pd.DataFrame(index=['Calcasieu', 'Jefferson', 'Orleans', 'St. Bernard', 'St. Tammany'], columns=['Pipeline Option 1 count'], data=[12717, 25128, 41638, 7848, 10939])
print('Comparison of dataset and pipeline Option 1 counts for top 5 parishes by number of properties that participated in Road Home')
dataset_pipeline_compare = pd.concat([
                                        rh[
                                            (rh['PARISH'].isin(['Orleans', 'Jefferson', 'Calcasieu', 'St. Tammany', 'St. Bernard'])) & 
                                            (rh['Closing Option']==1)
                                            ]
                                            .groupby('PARISH')
                                            .agg(
                                                {
                                                    'Total CG Amount' : 'count', 
                                                    }
                                                )
                                            .rename(columns = {'Total CG Amount' : 'Dataset Option 1 count'})
                                        ,
                                        pipeline
                                        ],
                                        axis=1
                                    )
dataset_pipeline_compare['Difference'] = dataset_pipeline_compare['Dataset Option 1 count']-dataset_pipeline_compare['Pipeline Option 1 count']
display(dataset_pipeline_compare)

Comparison of dataset and pipeline Option 1 counts for top 5 parishes by number of properties that participated in Road Home


,Dataset Option 1 count,Pipeline Option 1 count,Difference
Calcasieu,12702,12717,-15
Jefferson,25148,25128,20
Orleans,41614,41638,-24
St. Bernard,7852,7848,4
St. Tammany,10944,10939,5


Setting up rh_clean, which will exclude records from the analysis. The criteria for inclusion are:
- rh['Current Damage Assessment']>0 : Exclude records with no recorded damage. This excludes properties that did not have damage recorded.
- rh['Total CG Amount']>0 : Exclude records with no Compensation Grants. This excludes properties that may have damage recorded but did not get repair/rebuilding grants.
- rh['Closing Option']==1 : Exclude any properties whose owners chose Option 2 (sell property to the state and buy another home in Louisiana) and Option 3 (sell property to the state and do not buy a home in Louisiana).
- rh['Closed File - Option 2/3, Sold Home, Ineligible, Deceased, and Closed with RHEI and IMM Only']=='N' : This flag was created by the state. This includes the three criteria listed above as well as edge cases that might skew the data. Earlier criteria are included to ensure all matching properties are excluded.
- rh['Closed with Unmet Needs (Y/N)']=='N' : Exclude properties that got money through the Unmet Needs program. These properties received additional money from the state for problems outside of their control (such as contractors taking money but not doing work). The state did not provide specific Unmet Needs amounts, so these properties must be excluded entirely. 
- rh['ARS File (Yes/No)=='N'] : Exclude properties that are subject to a lawsuit between the state and its contractor (ICF) over incorrect grant values


In [8]:
rh_clean = rh[
    (rh['Current Damage Assessment']>0) & 
    (rh['Total CG Amount']>0) & 
    (rh['Closing Option']==1) &
    (rh['Closed File - Option 2/3, Sold Home, Ineligible, Deceased, and Closed with RHEI and IMM Only']=='N') &
    (rh['Closed with Approved Unmet Needs (Y/N)']=='N') &
    (rh['ARS File (Yes/No)']=='N')
    ].copy()

In [9]:
len(rh[(rh['Closed with Approved Unmet Needs (Y/N)']=='Y') & (rh['Closing Option']==1)])

6152

In [10]:
print('Basic stats (with non-relevant records excluded):')
print(f"Number of records: {len(rh_clean):,.0f}")
print(f"Number of non-null records: {len(rh_clean.dropna(axis=1)):,.0f}")
print(f"Number of records where CG>0 and DA>0: {len(rh_clean[(rh_clean['Current Damage Assessment']>0) & (rh_clean['Total CG Amount']>0)]):,.0f}")
print(f"Total damage assessment: ${rh_clean['Current Damage Assessment'].sum():,.2f}")
print(f"Total grant amount: ${rh_clean['TOTAL_CLOSING_AMOUNT'].sum():,.2f}")
print(f"Total CG amount: ${rh_clean['Total CG Amount'].sum():,.2f}")
print(f"Total ACG amount: ${rh_clean['Total ACG Amunt'].sum():,.2f}")
print(f"Total elevation amount: ${rh_clean['Total Elevation Amount'].sum():,.2f}")
print(f"Total IMM amount: ${rh_clean['Total IMM Amount'].sum():,.2f}")
print(f"Total DOB amount: ${rh_clean['Current Total DOB Amount (no Legal Fees removed)'].sum():,.2f}")
print(f"Total legal fees: ${rh_clean['Current Legal Fees'].sum():,.2f}")


Basic stats (with non-relevant records excluded):
Number of records: 93,490
Number of non-null records: 93,490
Number of records where CG>0 and DA>0: 93,490
Total damage assessment: $12,378,566,488.50
Total grant amount: $6,438,287,932.89
Total CG amount: $4,127,228,647.02
Total ACG amount: $1,517,663,016.58
Total elevation amount: $636,665,696.75
Total IMM amount: $156,730,572.54
Total DOB amount: $3,910,966,433.69
Total legal fees: $26,839,981.72


In some cases, the DOB amount increased after the initial grants were awarded. In those cases, the combined CG/ACG amount in these records was not adjusted to reflect the increase. However, the state sought to claw back the money from CG/ACG, which now represents an overpayment.

As a result, we have to adjust the combined CG/ACG values so they reflect the appropriate amount actually provided to the property owner.

In [11]:
rh_clean['Current DOB (no legal fees)'] = rh_clean['Current Total DOB Amount (no Legal Fees removed)'].fillna(0.0)-rh_clean['Current Legal Fees'].fillna(0.0)

rh_clean['dob_difference'] = rh_clean['Current DOB (no legal fees)'] - rh_clean['Closing Total DOB Amount']
print(f"DOB increase : {len(rh_clean[rh_clean['dob_difference']>0]):,.0f}")
print(f"DOB decrease : {len(rh_clean[rh_clean['dob_difference']<0]):,.0f}")

DOB increase : 12,819
DOB decrease : 14,105


According to Jeff Haley, DOB could be adjusted up or down through the process. A DOB decrease, however, would already be accounted for in the Current DOB figure and the Current CG/ACG figures. So we want to only subtract adjustments where DOB increased (in other words, where dob_difference>0).

Subtracting values from CG. A version of this analysis with the DOB Increases subtracted from ACG yields almost identical results.

In [12]:
rh_clean.loc[rh_clean['dob_difference']>0, 'Total CG Amount'] = rh_clean['Total CG Amount']-rh_clean['dob_difference']

In [13]:
rh_clean['compensation'] = rh_clean['Total CG Amount'].fillna(0.0)+rh_clean['Total ACG Amunt'].fillna(0.0)+rh_clean['Current DOB (no legal fees)'].fillna(0.0)

In [14]:
rh_clean[(rh_clean['compensation']==rh_clean['Current Damage Assessment'])][['Census Block', 'Total CG Amount', 'Total ACG Amunt', 'compensation', 'Current Damage Assessment', 'Closing Damage Assessment', 'Current Total DOB Amount (no Legal Fees removed)', 'Closing Total DOB Amount']]

,Census Block,Total CG Amount,Total ACG Amunt,compensation,Current Damage Assessment,Closing Damage Assessment,Current Total DOB Amount (no Legal Fees removed),Closing Total DOB Amount
6,220710081021016,"11,392.28",0.00,"66,370.80","66,370.80","66,370.80","54,978.52","54,978.52"
8,220710121012015,"65,536.32",0.00,"94,533.80","94,533.80","94,533.80","28,997.48","29,483.26"
16,220510208002007,"7,905.23",0.00,"17,545.14","17,545.14","17,545.14","9,639.91","9,639.91"
17,220510234003008,"19,087.22",0.00,"36,751.91","36,751.91","36,751.91","17,664.69","17,664.69"
19,220710006041005,"30,408.52",0.00,"51,723.71","51,723.71","21,667.22","21,315.19","25,780.19"
...,...,...,...,...,...,...,...,...
130046,220710017401003,"9,193.44",0.00,"155,280.46","155,280.46","155,280.46","146,087.02","146,087.02"
130047,220550019011007,"8,151.26","32,098.74","40,250.00","40,250.00","40,250.00",0.00,0.00
130049,221059547001048,"5,856.87",0.00,"12,616.80","12,616.80","11,984.70","6,759.93","6,759.93"
130050,221059548003006,"12,488.28","31,255.35","45,850.00","45,850.00","45,850.00","2,106.37","2,106.37"


In [15]:
rh_clean[(rh_clean['compensation']>rh_clean['Current Damage Assessment'])]

,Structure Type,GIS City,GIS State,GIS Zip,PARISH,Closing Option,TOTAL_CLOSING_AMOUNT,Total CG Amount,Total ACG Amunt,Total Elevation Amount,Total IMM Amount,NOLA Planning District Number,NOLA Planning District Name,NOLA Neighborhood Number,NOLA Neighborhood Name,STFID,Census Block,Block Group,Census Track,ORL_CD_NUM,ORL_CD_NAM,Current Damage Assessment,Current Damage Assessment - Type 1,Current Damage Assessment - Type 2,Damage Type 1 or 2,Current PSV,Current Total DOB Amount (no Legal Fees removed),Current Legal Fees,Closing Damage Assessment,PSV at Closing,"Closed file - PSV Option 1 With No ACG -excluding Sold Home, Inelig, Deceased and files closed with RHEI & IMM only -23,912","Closed File - Option 2/3, Sold Home, Ineligible, Deceased, and Closed with RHEI and IMM Only",Closed with Approved Unmet Needs (Y/N),Applicant With Current Insurance (Private and/or Flood) Y/N,Closing Total DOB Amount,Difference between Current PSV and Current Damage Assessment Type 2 (col AG - col AC,ARS File (Yes/No),Current DOB (no legal fees),dob_difference,compensation
7,Single (including mobile home),HARVEY,LA,70058-5411,Jefferson,1.00,"6,358.26","6,358.26",0.00,0.00,0.00,NaN,NaN,NaN,NaN,220510278052010,220510278052010,220510278052,22051027805,NaN,NaN,"27,263.19","302,889.00","27,263.19",2.00,"138,000.00","20,904.93",0.00,"27,263.19","138,000.00",N,N,N,Y,"20,904.93","-110,736.81",N,"20,904.93",0.00,"27,263.19"
10,Single (including mobile home),METAIRIE,LA,70005-4008,Jefferson,1.00,"41,715.80","29,571.80",0.00,0.00,"7,500.00",NaN,NaN,NaN,NaN,220510226001005,220510226001005,220510226001,22051022600,NaN,NaN,"60,369.21","661,041.60","60,369.21",2.00,"748,000.00","35,446.41","4,649.00","60,369.21","748,000.00",N,N,N,Y,"26,153.41","-687,630.79",N,"30,797.41","4,644.00","60,369.21"
11,Single (including mobile home),AVONDALE,LA,70094-2423,Jefferson,1.00,"38,906.51","21,984.56","9,421.95",0.00,"7,500.00",NaN,NaN,NaN,NaN,220510275025004,220510275025004,220510275025,22051027502,NaN,NaN,"31,406.51","106,800.00","31,406.51",2.00,"98,000.00",0.00,0.00,"31,406.51","98,000.00",N,N,N,N,"8,414.42","-66,593.49",N,0.00,"-8,414.42","31,406.51"
75,Single (including mobile home),METAIRIE,LA,70006-3919,Jefferson,1.00,"12,240.60","12,240.60",0.00,0.00,0.00,NaN,NaN,NaN,NaN,220510217006003,220510217006003,220510217006,22051021700,NaN,NaN,"72,944.40","240,832.20","72,944.40",2.00,"210,000.00","60,703.80",0.00,"72,944.40","175,000.00",N,N,N,Y,"66,387.58","-137,055.60",N,"60,703.80","-5,683.78","72,944.40"
99,Single (including mobile home),METAIRIE,LA,70003-2107,Jefferson,1.00,"10,224.08","10,224.08",0.00,0.00,0.00,NaN,NaN,NaN,NaN,220510234001020,220510234001020,220510234001,22051023400,NaN,NaN,"99,495.54","270,269.40","99,495.54",2.00,"198,000.00","89,271.46",0.00,"99,495.54","198,000.00",N,N,N,Y,"89,271.46","-98,504.46",N,"89,271.46",0.00,"99,495.54"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129982,Duplex (with one owner-occupied unit),NEW ORLEANS,LA,70119-1332,Orleans,1.00,"70,325.90","49,207.13","21,118.77",0.00,0.00,4,Mid-City,14,Seventh Ward,220710030001000,220710030001000,220710030001,22071003000,NaN,NaN,"110,295.90","266,026.20","110,295.90",2.00,"128,000.00","40,000.00",0.00,"110,295.90","128,000.00",N,N,N,Y,"40,000.00","-17,704.10",N,"40,000.00",0.00,"110,325.90"
129996,Single (including mobile home),NEW ORLEANS,LA,70131-8319,Orleans,1.00,"12,475.63","4,975.63",0.00,0.00,"7,500.00",12,Algiers,9,Tall Timbers/Brechtel,220710006142007,220710006142007,220710006142,22071000614,NaN,NaN,"18,472.42","499,667.40","18,472.42",2.00,"175,000.00","13,496.79",0.00,"18,472.42","175,000.00",N,N,N,Y,"13,496.79","-156,527.58",N,"13,496.79",0.00,"18,472.42"
129997,Single (including mobile home),SULPHUR,LA,70663-5072,Calcasieu,1.00,"109,313.93","50,679.26","28,634.67","30,000.00",0.00,NaN,NaN,NaN,NaN,220190030002005,220190030002005,220190030002,22019003000,NaN,

Filter out records where compensation is higher than damage

In [16]:
rh_clean = rh_clean[(rh_clean['compensation'].round(2)<=rh_clean['Current Damage Assessment'].round(2))]
rh_clean.loc[rh_clean['Applicant With Current Insurance (Private and/or Flood) Y/N']=='N', 'insurance_penalty'] = ((rh_clean['Total CG Amount']*100)/70)-rh_clean['Total CG Amount']
# rh_clean = rh_clean[(rh_clean['compensation']<=rh_clean['Current Damage Assessment']+0.000001)]

In [26]:
## WITH FILTERED DATASET
len(rh_clean[rh_clean['compensation']<rh_clean['Current Damage Assessment']])

46687

In [36]:
46687 / len(rh_clean)

0.5087336958298373

In [ ]:
## UNFILTERED DATASET

In [28]:
rh['Current DOB (no legal fees)'] = rh['Current Total DOB Amount (no Legal Fees removed)'].fillna(0.0)-rh['Current Legal Fees'].fillna(0.0)

In [29]:
rh['dob_difference'] = rh['Current DOB (no legal fees)'] - rh['Closing Total DOB Amount']

In [31]:
# Only subtract the difference from the CG amount if the Current DOB is higher than the Closing DOB
# This is because if the DOB decreased it would not have an effect on the CG
rh.loc[rh['dob_difference']>0, 'Total CG Amount'] = rh['Total CG Amount']-rh['dob_difference']

In [33]:
rh['compensation'] = rh['Total CG Amount']+rh['Total ACG Amunt']+rh['Current DOB (no legal fees)']


In [34]:
len(rh[(rh['Closing Option']==1) & (rh['compensation']<rh['Current Damage Assessment'])])

57468

In [37]:
rh_clean['uncovered'] = (rh_clean['Current Damage Assessment']-rh_clean['compensation'])/rh_clean['Current Damage Assessment']

In [38]:
pd.concat([rh_clean[rh_clean['uncovered']>0].groupby('PARISH').size(), rh_clean.groupby('PARISH').size()], axis=1).apply(lambda x: x[0]/x[1], axis=1).sort_values()

PARISH
Lafayette              0.18
St. Helena             0.22
St. John the Baptist   0.24
St. Landry             0.26
Ascension              0.26
Livingston             0.26
East Feliciana         0.27
Iberville              0.27
St. Martin             0.28
Sabine                 0.28
East Baton Rouge       0.29
Tangipahoa             0.31
Pointe Coupee          0.31
Assumption             0.31
Evangeline             0.31
St. Charles            0.34
Washington             0.35
Lafourche              0.35
Acadia                 0.35
St. James              0.35
Calcasieu              0.36
St. Mary               0.36
Jefferson              0.37
Beauregard             0.38
Terrebonne             0.38
Vernon                 0.40
Jefferson Davis        0.42
Allen                  0.42
St. Tammany            0.46
Iberia                 0.49
West Feliciana         0.50
West Baton Rouge       0.50
Plaquemines            0.54
Vermilion              0.60
Orleans                0.67
Cameron      

In [39]:
len(rh_clean)

91771